##Les imports nécessaires

Après avoir installer deeplake, il faudra cliquer sur restart runtime et relancer la première cellule, sinon le dataset ne se charge pas.

In [ ]:
!pip install deeplake

In [ ]:
import sys
import numpy as np
import cv2
import time
import skimage.io
from scipy.signal import medfilt2d
from skimage import metrics 
from skimage.color import rgb2lab
from skimage.color import lab2rgb
from skimage.morphology import disk, square, erosion, reconstruction
from scipy.io import loadmat
from google.colab.patches import cv2_imshow


###Pour visualiser le dataset

In [ ]:
import deeplake
ds = deeplake.load("hub://activeloop/glas-test")
ds.visualize()

## Function de PCA


In [ ]:
def new_pca_color(image, num_channels):
    # Convert the image to a floating point array
    image = image.astype(np.float32)
    
    # Reshape the image array to a 2D array of size (num_pixels, num_channels)
    num_pixels = image.shape[0] * image.shape[1]
    image_2d = image.reshape(num_pixels, image.shape[2])
    
    # Subtract the mean of each channel from the image array
    image_mean = np.mean(image_2d, axis=0)
    image_2d = image_2d - image_mean
    
    # Compute the covariance matrix of the image array
    image_cov = np.cov(image_2d.T)
    
    # Compute the eigenvalues and eigenvectors of the covariance matrix
    eigvals, eigvecs = np.linalg.eig(image_cov)
    
    # Sort the eigenvectors by their corresponding eigenvalues in descending order
    eigvals_sorted_indices = np.argsort(eigvals)[::-1]
    eigvecs_sorted = eigvecs[:, eigvals_sorted_indices]
    
    # Select the top num_channels eigenvectors as the basis for the reduced color space
    basis = eigvecs_sorted[:, :num_channels]
    
    # Project the image array onto the reduced color space using the selected basis
    image_projected = np.dot(image_2d, basis)
    
    # Add the mean of each channel back to the projected image
    image_projected += image_mean
    
    # Reshape the projected image back to its original size and format
    image_reduced = image_projected.reshape(image.shape[0], image.shape[1], num_channels)
    image_reduced = image_reduced.astype(np.uint8)
    
    return image_reduced

##Fonction pour la reconstruction morphologique RGB

In [ ]:
def w_ColorRecons_CO(f, radius):
    #se = square(radius)
    se = square(radius)
    if len(f.shape) < 3:
        print('Please input a color image!')
    else:
        # Convert f to double and extract channels
        f = np.double(f)
        f_r = f[:,:,0]
        f_g = f[:,:,1]
        f_b = f[:,:,2]
        
        # Apply PCA and extract channels
        f_pca = np.double((new_pca_color(f,3)))
        
        f1 = f_pca[:,:,0]
        f2 = f_pca[:,:,1]
        f3 = f_pca[:,:,2]
        # Transform data
        data1 = f1*10**3 + f2*10 + f3 + f_r*10**-3 + f_g*10**-6 + f_b*10**-9
        Max1 = np.max(data1)
        data2 = f1*10**3 + f2*10 + f3 + f_g*10**-3 + f_r*10**-6 + f_b*10**-9
        Max2 = np.max(data2)
        
        data3 = f1*10**3 + f2*10 + f3 + f_b*10**-3 + f_r*10**-6 + f_g*10**-9
        Max3 = np.max(data3)
        
        # Process data
        imput_data1 = erosion(data1, se)
        imput_data2 = erosion(data2, se)
        imput_data3 = erosion(data3, se)
        
        f_rec1 = reconstruction(imput_data1, data1)
        f_rec2 = reconstruction(imput_data2, data2)
        f_rec3 = reconstruction(imput_data3, data3)
        
        imput2_data1 = erosion(Max1-f_rec1, se)
        imput2_data2 = erosion(Max2-f_rec2, se)
        imput2_data3 = erosion(Max3-f_rec3, se)
        
        f_g1 = Max1 - reconstruction(imput2_data1, Max1-f_rec1)
        f_g2 = Max2 - reconstruction(imput2_data2, Max2-f_rec2)
        f_g3 = Max3 - reconstruction(imput2_data3, Max3-f_rec3)
        
    # Return to RGB format
    tt1 = f_g1 - np.floor(f_g1)
    tt2 = f_g2 - np.floor(f_g2)
    tt3 = f_g3 - np.floor(f_g3)
    
    g1_r = np.floor(tt1*10**3)

    g1_g = np.floor(tt2*10**3)
    g1_b = np.floor(tt3*10**3)
    
    output_f = np.dstack((np.uint8(g1_b), np.uint8(g1_g), np.uint8(g1_r)))
    return output_f

Fonction pour afficher la segmentation après avoir fait le clustering

In [ ]:
def initfcm(cluster_n, row_col):
    # Initialize the fuzzy partition matrix (U) with random values
    U = np.random.rand(cluster_n, row_col)
    # Normalize the rows of U so that they add up to 1
    U /= np.sum(U, axis=0)
    return U

def Label_image(f, L):
    # fs is the result of segmentation, center_p is the center pixel of each
    # areas
    # f is the original image
    # L is the segmented image using waterhsed transformation
    f = f.astype(float)
    num_area = int(L.max()) # The number of segmented areas
    center_lab = np.zeros((num_area, 3))
    Num_p = np.zeros(num_area)
    if f.ndim < 3:
        M, N = f.shape
        s3 = L
        fs = np.zeros((M, N))
        center_p = np.zeros((num_area, 1))
        for i in range(1, num_area+1):
            f2 = f[s3 == i]
            f_med = np.median(f2)
            fx = (s3 == i) * f_med
            fs = fs + fx
            center_p[i-1, :] = np.uint8(f_med)
            Num_p = np.zeros(num_area)
        fs = np.uint8(fs)
    else:  # Color image
        M, N = f[:, :, 0].shape
        s3 = L
        fs = np.zeros((M, N, 3))
        center_p = np.zeros((num_area, 3))
        for i in range(1, num_area+1):
            fr2 = f[:, :, 0][s3 == i]
            r_med = np.median(fr2)
            r = (s3 == i) * r_med
            fg2 = f[:, :, 1][s3 == i]
            g_med = np.median(fg2)
            g = (s3 == i) * g_med
            fb2 = f[:, :, 2][s3 == i]
            b_med = np.median(fb2)
            b = (s3 == i) * b_med
            fs = fs + np.stack((r, g, b), axis=2)
            center_p[i-1, :] = np.uint8([r_med, g_med, b_med])
            Num_p[i-1] = (s3 == i).sum()
        fs = np.uint8(fs)
    TT = np.stack((center_p[:, 0], center_p[:, 1], center_p[:, 2]), axis=1)
    TT2 = rgb2lab(TT)
    TT2r = TT2[:, 0]
    TT2g = TT2[:,  1]
    TT2b = TT2[:,  2]
    center_lab[:, 0] = TT2r.flatten()
    center_lab[:, 1] = TT2g.flatten()
    center_lab[:, 2] = TT2b.flatten()
    return fs, center_p, Num_p, center_lab
def fcm_image_color(f, U):
    m, n = f.shape[:2]
    U = U.T
    idx_f = np.zeros(m * n)
    for i in range(m * n):
        x = U[i]
        idx = np.where(x == max(x))[0]
        idx = idx[0]
        idx_f[i] = idx
    imput_f = idx_f.reshape((m, n))
    #gx = Label_image(f, imput_f)
    return imput_f
# center[k,:] = sum(mf[:,k].T.dot(data) )/ sum(mf[:,k])

##Fonction pour FRFCM

In [ ]:


def FRFCM_c(data, cluster_n, radius, w_size, options=None):
    # Firstly, we use multivariate morphological reconstruction to reconstruct original image to suppress noise;
    # Secondly, we implement FCM;
    # Thirdly, we use a median filter to smooth the fuzzy membership U;
    # Finally, we normalize U;
    # Input variants and parameters
    # data is a 3D data, it means that the input is a color image 
    # cluster_n denotes the number of cluster centers
    # radius denotes the parameter of structuring element used in
    # mrophological recosntruction
    # w_size is the scale of the filtering window 
    eps = sys.float_info.epsilon
    if options is None:
        # Change the following to set default options
        options = [2,  # exponent for the partition matrix U
                   100,  # max. number of iteration
                   1e-5,  # min. amount of improvement
                   1]  # info display during iteration

    # If "options" is not fully specified, pad it with default values.
    """if len(options) < 4:
        tmp = [None] * 4
        tmp[:len(options)] = options
        options = tmp

    # If some entries of "options" are nan's, replace them with defaults.
    nan_index = np.where(np.isnan(options))[0]
    options[nan_index] = [2, 100, 1e-5, 0][nan_index]
    if options[0] <= 1:
        raise ValueError("The exponent should be greater than 1!")
    """
    expo = options[0]  # Exponent for U
    max_iter = options[1]  # Max. iteration
    min_impro = options[2]  # Min. improvement
    display = options[3]  # Display info or not
    obj_fcn = np.zeros(max_iter) # Array for objective function
    
    # step 1, morphological reconstruction
    data_rgb = w_ColorRecons_CO(data, radius)  # radius means maximal radius
    
  
    data_lab = rgb2lab(data_rgb)
    # step 2, FCM on histogram
    data_l = data_lab[:,:,0]
    data_a = data_lab[:,:,1]
    data_b = data_lab[:,:,2]
    data = np.vstack((data_l.flatten(), data_a.flatten(), data_b.flatten())).T
    row, col = data_a.shape
    U = np.random.rand(row*col, cluster_n)
    U /= np.sum(U, axis=0)
    Uc = [U]
    center = np.zeros((cluster_n,3))
    for i in range(max_iter):
        # MF matrix after exponential modification
        mf = U**expo
        
        nom = mf.T@data
        den = sum(mf)
        den = den.reshape((cluster_n, 1))
        center= np.divide(nom, den)

        center_l = center[:, 0]
        center_a = center[:, 1]
        center_b = center[:, 2]
        center_lab = np.dstack((center_l, center_a, center_b))

        rgb_center = 255 * lab2rgb(center_lab)
        out = np.zeros((center.shape[0] , data.shape[0]))
        if center.shape[1] > 1:
            for k in range(center.shape[0]):
                out[k, :] = np.sqrt(np.sum((data - np.ones((data.shape[0], 1)) *center[k,:])**2, axis=1))
                
        else:  # 1-D data
            for k in range(center.shape[0]):
                out[k, :] = abs(center[k]-data).T
        dist = out + eps

        


        tmp = dist**(-2/(expo-1))

        U = tmp /  np.sum((tmp) + np.finfo(float).eps,axis=0)
        U = np.transpose(U)

        Uc.append(U)
        
        # objective function
        if i > 0:
            # if abs(obj_fcn[i] - obj_fcn[i-1])/obj_fcn[i] < min_impro, break
            if (np.max(np.max(np.abs(Uc[i]-Uc[i-1])))) < min_impro:
                break
    

    iter_n = i  # Actual number of iterations
    #obj_fcn[iter_n+1:max_iter] = []
    GG = np.zeros((center.shape[0], row * col))
    
    for k3 in range(center.shape[0]):
        U1 = U[:,k3]
        U1 = np.reshape(U1, (row, col))
        UU = medfilt2d(U1, [w_size, w_size])

        GG[k3,:] = UU.flatten()

    U = GG / (np.sum(GG) + eps)
    # Normalization
    center_l = center[:, 0]
    center_a = center[:, 1]
    center_b = center[:, 2]
    center_lab = np.dstack((center_l, center_a, center_b))

    center = 255 * lab2rgb(center_lab)
    return [center, U, obj_fcn,iter_n]

## Fonction pour calculer les métriques

## steps_Fcm_Seg pour automatiser dans le main

In [ ]:
def my_metrics(segmented, ground_truth):
    # Calculate true positive (TP), false positive (FP), and false negative (FN)
    TP = np.sum(segmented == ground_truth)
    FP = np.sum(np.logical_and(segmented != 0, ground_truth == 0)  )
    FN = np.sum(np.logical_and(segmented == 0, ground_truth != 0))
    h,w= segmented.shape[0],segmented.shape[1]
    S = np.sum(segmented == ground_truth)/(h*w)
    # Calculate precision and recall
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall)

    return np.array([precision, recall, f1_score, S])

In [ ]:
def steps_fcm_seg(f_ori, cluster, se, w_size):
  #f_ori = f_ori.astype(np.uint8) # convert to uint8 to prevent lossy conversion when saving image
  if len(f_ori.shape) == 2:
    f_ori= cv2.cvtColor(f_ori, cv2.COLOR_GRAY2RGB)
  f_ori = f_ori.astype(np.float64)
  start = time.time()
  center, U1, _, iterations = FRFCM_c(f_ori, cluster, se, w_size)

  end = time.time()
  print(f'running time is: {end - start}.\n{iterations} iterations')

  f_seg = fcm_image_color(f_ori, U1)
  return f_seg, end - start, iterations


##Main

In [ ]:
import glob
import tensorflow as tf 

cluster = 2
se = 3 # the parameter of structuring element used for morphological reconstruction
w_size = 3 #Window size
gloabl_metrics= np.zeros((len(ds["images"]),4))
global_times= np.zeros((len(ds["images"]),3))

for i in range(len(ds["images"])):
  cluster = 2
  img = tf.constant(ds["images"][i])
  img = np.array(img)
  cv2_imshow(img)
  img1 = tf.constant(ds["masks"][i])
  img1 = np.array(img1).astype(np.uint8)
  img1[img1 == 1] = 255
  cv2_imshow(img1)
  f_ori1 = img
  #cv2_imshow(f_ori1)
  #cv2_imshow(img1)
  
  max_metrics= np.zeros((4,))

  while cluster <= 5:
    
    print("Number of clusters : ",cluster)
    ### Segmentation of color image
    #f_ori = skimage.io.imread(f'12003.jpg')
    
    #f_ori1= cv2.cvtColor(f_ori, cv2.COLOR_BGR2RGB)
    img,time1,it = steps_fcm_seg(f_ori1, cluster, se, w_size)
    img_seg= Label_image(f_ori1,img)[0]
    
    ####Pour calculer les metriques  enlever ces commentaires
    """img_seg = cv2.cvtColor(img_seg, cv2.COLOR_BGR2GRAY)
    img_seg[img_seg != 0] = 255
    my_metrics(img_seg,img1[:,:,0])
    metrics = my_metrics(img_seg,img1[:,:,0])
    if metrics[2] > max_metrics[2]:
      max_metrics = metrics
      global_times[i][0] = time1
      global_times[i][1] = it
      global_times[i][2] = cluster"""
    cv2_imshow(img_seg)
    cluster += 1


  gloabl_metrics[i] = max_metrics
